# Arbol de decision
En este notebook vamos a entrenar y explorar un arbol de decision.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, tree
from ipywidgets import Button, IntSlider, interactive

from preprocessing import cargarDatasets
from preprocessing import prepararSetDeEntrenamiento
from preprocessing import prepararSetDeValidacion
from preprocessing import ingenieriaDeFeauturesArboles1

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split,GridSearchCV

## Carga y preprocesamiento de los datos
Cargamos los datasets y los preparamos para usar.

In [2]:
train_df,final_df = cargarDatasets()
train_df = prepararSetDeEntrenamiento(train_df)
final_df = prepararSetDeValidacion(final_df)

Ahora que ya tenemos los datasets cargados, **le aplicamos al dataset de entrenamiento distintos preprocesamientos, y luego lo particionamos usando k-fold cross validation.** a chequear esto ultimo

In [3]:
X,y,df,y_encoder = ingenieriaDeFeauturesArboles1(train_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y)

* ingenieria de feautures (2 formas)
* dividimos cada set en 3.
* entrenamos (2 arboles)
* metricas

fit: ajusta el modelo al set de entrenamiento
predict: que agarre un ejemplo y haga una prediccion (predice la clase a la que pertenece)


hiperparametros: quiero encontrar el mejor modelo para este dataset. Son parametros que el modelo no aprende, hay que decirselos.
-> random state: quiero que si un valor es random, que sea reproducible. Que el modelo vuelva a dar lo mismo cada vez que lo corro (que sea determinista).
->split: cuantos feautures miro para ir dividiendo.

In [11]:
params2 = {
    'max_depth': np.arange(1, 10),
    'min_samples_leaf': np.arange(1, 16),
    "criterion": ["gini", "entropy"],
}

arbol_2 = tree.DecisionTreeClassifier()
gscv2 = GridSearchCV(
    arbol_2, params2, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

arbol_2.fit(X_train, y_train)
y_pred2 = arbol_2.predict(X_test)
accuracy_score(y_test, y_pred2)

0.815474362910654

In [12]:
print(f"Best score: {gscv2.best_score_}")
print(f"Best params {gscv2.best_params_}")

Best score: 0.8569477829092987
Best params {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 3}
